In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['data/names/Italian.txt', 'data/names/Korean.txt', 'data/names/French.txt', 'data/names/Chinese.txt', 'data/names/Portuguese.txt', 'data/names/Japanese.txt', 'data/names/Vietnamese.txt', 'data/names/Polish.txt', 'data/names/Russian.txt', 'data/names/Dutch.txt', 'data/names/Scottish.txt', 'data/names/Spanish.txt', 'data/names/Arabic.txt', 'data/names/Irish.txt', 'data/names/Czech.txt', 'data/names/English.txt', 'data/names/German.txt', 'data/names/Greek.txt']
Slusarski


In [2]:
category_lines[all_categories[1]][:5]

['Ahn', 'Baik', 'Bang', 'Byon', 'Cha']

In [3]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
torch.Size([5, 1, 57])


In [4]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [5]:
n_hidden = 128
rnn = RNN(n_letters,n_hidden, n_categories)

In [6]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)
output, next_hidden  = rnn(input,hidden)


In [7]:
output, next_hidden  = rnn(input,next_hidden)

In [8]:
output, next_hidden  = rnn(input,next_hidden)

In [9]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)


In [10]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Arabic', 12)


In [11]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Czech / line = Samz
category = Spanish / line = Abasolo
category = Dutch / line = Simon
category = Spanish / line = Tomas
category = Vietnamese / line = Luong
category = Spanish / line = Abarca
category = English / line = Yeung
category = Portuguese / line = Rosario
category = Dutch / line = Vennen
category = Vietnamese / line = Pham


In [12]:
criterion = nn.NLLLoss()

In [16]:
learning_rate = 0.008 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        print(output)
    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

category, line, category_tensor, line_tensor = randomTrainingExample()
train(category_tensor,line_tensor)

tensor([[-2.9557, -2.8932, -2.9286, -2.8535, -2.9075, -2.9663, -2.9591,
         -2.9305, -2.8612, -2.8368, -2.8717, -2.8751, -2.8688, -2.7894,
         -2.8631, -2.8781, -2.9145, -2.8915]])
tensor([[-2.9243, -2.7888, -3.0375, -2.9106, -2.9079, -3.0113, -2.8782,
         -2.9312, -2.9323, -2.8990, -2.8017, -2.8035, -2.8486, -2.8818,
         -2.8699, -2.9464, -2.8589, -2.8330]])
tensor([[-2.9283, -2.9449, -2.9855, -2.8316, -2.9326, -2.9225, -2.9999,
         -2.8890, -2.9566, -2.7941, -2.8612, -2.7988, -2.8762, -2.7794,
         -2.8637, -2.8003, -2.9334, -2.9708]])
tensor([[-2.9879, -2.7912, -2.9469, -2.8955, -2.8486, -2.8853, -2.9184,
         -2.9923, -2.9489, -2.7726, -2.8763, -2.8483, -2.8607, -2.7894,
         -2.8393, -2.9399, -3.0167, -2.9121]])


(tensor([[-2.9879, -2.7912, -2.9469, -2.8955, -2.8486, -2.8853, -2.9184,
          -2.9923, -2.9489, -2.7726, -2.8763, -2.8483, -2.8607, -2.7894,
          -2.8393, -2.9399, -3.0167, -2.9121]]), 2.918396234512329)

In [ ]:
import time
import math

n_iters = 50000
print_every = 500
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Size([1])
torch.Size([1, 18]) torch.Si

In [14]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(all_losses)
plt.show()

In [22]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 1000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [19]:
n_categories

18

In [46]:
import torch.nn as nn

class RNN_test(nn.Module):
    def __init__(self, input_size,output_size):
        super(RNN_test, self).__init__()
        
        self.i2o = nn.Linear(input_size, output_size)

    def forward(self, input):
        print(input)
        output = self.i2o(input)
        return output


rn = RNN_test(2,3)
    

In [53]:
in_1 = torch.randn(1,2)
in_ = torch.tensor([10.,2.])
out = rn(in_)
print(out)

tensor([ 10.,   2.])
tensor([ 1.3127, -4.7569, -4.9580])
